In [ ]:
#!pip install paddlenlp

In [ ]:
# 用的paddlenlp的word embeddings，具体看 https://github.com/PaddlePaddle/PaddleNLP/blob/develop/docs/model_zoo/embeddings.md
# data封装类型还没弄懂 把batch封装好了就能train了

In [ ]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data
import timeit
from collections import Counter
from tqdm import tqdm

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
tr_in = open("couplet/train/in.txt",encoding='utf8').read()
tr_out = open("couplet/train/out.txt",encoding='utf8').read()
te_in = open("couplet/test/in.txt",encoding='utf8').read()
te_out = open("couplet/test/out.txt",encoding='utf8').read()

In [ ]:
test = list(zip(te_in.split('\n')[:-1], te_out.split('\n')[:-1]))
dev =  list(zip(tr_in.split('\n')[-3000:], tr_out.split('\n')[-3000:]))
train = list(zip(tr_in.split('\n')[:-3000], tr_out.split('\n')[:-3000]))

In [ ]:
len(tr_in.split('\n')),len(train),len(dev),len(test)

In [ ]:
from paddlenlp.embeddings import TokenEmbedding

token_embedding = TokenEmbedding(embedding_name="w2v.baidu_encyclopedia.target.word-word.dim300")

In [ ]:
vocab = list(set(tr_in + tr_out + te_in + te_out))

embeddings = dict()
for w in vocab:
    embeddings[w] = token_embedding.search(w).reshape(300,)

embeddings['<s>'] = np.random.rand(300,).astype('float32') *2 - 1 # range(-1:1)
embeddings['<e>'] = np.random.rand(300,).astype('float32') *2 - 1 # range(-1:1)
embeddings['<pad>'] = np.zeros(300,)
vocab.insert(0,'<e>')
vocab.insert(0,'<s>')
vocab.insert(0,'<pad>')
# creating word2id lookup dictionary
word2id = {w:i for i,w in enumerate(embeddings,start=3)}
word2id['<pad>'] = 0
word2id['<s>'] = 1
word2id['<e>'] = 2 
id2word = {i:w for w,i in word2id.items()}

max_sts_len = 20

len(vocab),len(word2id)

In [ ]:
id2word[0],id2word[1],id2word[2]

In [ ]:
sent = train[0][0]
['<s>'] + sent.split() + ['<e>']

In [ ]:
def prepare_x(sent,embeddings,max_sts_len=20): ##upper为输入的上联

    word_list = []
    #review = upper.strip().split(' ')
    review = ['<s>'] + sent.split() + ['<e>'] ##开头加符号1，结束加符号2
    
    if len(review) >= max_sts_len:
        review = review[:max_sts_len]
        origanal_len = max_sts_len
    else:
        origanal_len = len(review)
        for i in range(len(review), max_sts_len):
            review.append('<pad>') ## 词向量维度为200
            
    for word in review:                        
        embedding_vector = embeddings[word]
        word_list.append(embedding_vector)   

    #word_list.append([origanal_len for j in range(300)]) ## 最后一行元素为句子实际长度
    word_list = np.stack(word_list) 
    
    return word_list

In [ ]:
def prepare_y(sent,word2ix,max_sts_len=20):

    word_list = [1] ##开头加起始符号1
    for word in sent.split():
        word_idx = word2ix[word]
        word_list.append(word_idx)
    word_list.append(2) ##结束加终止符号2
    
    if len(word_list) >= max_sts_len:
        origanal_len = max_sts_len
        word_list = word_list[:max_sts_len]
    else:
        origanal_len = len(word_list)
        for i in range(len(word_list), max_sts_len):
            word_list.append(0) ## 不够长度则补0  
    #word_list.append(origanal_len) ##最后一个元素为句子长度
    
    return np.array(word_list)

In [ ]:
#[(i,len(k[0].split())) for i,k in enumerate(train) if len(k[0].split()) >= 20]
sent_x,sent_y = train[0][0],train[0][1]#,train[12864][0]
prepare_x(sent_x,embeddings).shape,sent_x

In [ ]:
prepare_y(sent_y,word2id),sent_y

In [ ]:
class Encoder(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, num_layers=2, dropout=0.2):
        super().__init__()
 
        self.embedding_dim = embedding_dim #词向量维度，本项目中是200维
        self.hidden_dim = hidden_dim #RNN隐层维度
        self.num_layers = num_layers #RNN层数
        self.dropout = dropout  #dropout
 
        self.rnn = nn.GRU(embedding_dim, hidden_dim,
                          num_layers=num_layers, dropout=dropout)
 
        self.dropout = nn.Dropout(dropout) #dropout层
 
    def forward(self, input_seqs, input_lengths, hidden=None):
        # src = [sent len, batch size]
        embedded = self.dropout(input_seqs)
        # embedded = [sent len, batch size, emb dim]
        #packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lengths) #将输入转换成torch中的pack格式，使得RNN输入的是真实长度的句子而非padding后的
        #outputs, hidden = self.rnn(packed, hidden)
        outputs, hidden = self.rnn(embedded)
        #outputs, output_lengths = torch.nn.utils.rnn.pad_packed_sequence(outputs)
        # outputs, hidden = self.rnn(embedded, hidden)
        # outputs = [sent len, batch size, hid dim * n directions]
        # hidden = [n layers, batch size, hid dim]
        # outputs are always from the last layer
        return outputs, hidden

In [ ]:
class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super(Attention, self).__init__()
        self.hidden_dim = hidden_dim
        self.attn = nn.Linear(self.hidden_dim * 2, hidden_dim)
        self.v = nn.Parameter(torch.rand(hidden_dim))
        self.v.data.normal_(mean=0, std=1. / np.sqrt(self.v.size(0)))
 
    def forward(self, hidden, encoder_outputs):
        #  encoder_outputs:(seq_len, batch_size, hidden_size)
        #  hidden:(num_layers * num_directions, batch_size, hidden_size)
        max_len = encoder_outputs.size(0)
        h = hidden[-1].repeat(max_len, 1, 1)
        # (seq_len, batch_size, hidden_size)
        attn_energies = self.score(h, encoder_outputs)  # compute attention score
        return F.softmax(attn_energies, dim=1)  # normalize with softmax
 
    def score(self, hidden, encoder_outputs):
        # (seq_len, batch_size, 2*hidden_size)-> (seq_len, batch_size, hidden_size)
        energy = torch.tanh(self.attn(torch.cat([hidden, encoder_outputs], 2)))
        energy = energy.permute(1, 2, 0)  # (batch_size, hidden_size, seq_len)
        v = self.v.repeat(encoder_outputs.size(1), 1).unsqueeze(1)  # (batch_size, 1, hidden_size)
        energy = torch.bmm(v, energy)  # (batch_size, 1, seq_len)
        return energy.squeeze(1)  # (batch_size, seq_len)

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, embedding_dim, hidden_dim, num_layers=2, dropout=0.2):
        super().__init__()
 
        self.embedding_dim = embedding_dim ##编码维度
        self.hid_dim = hidden_dim ##RNN隐层单元数
        self.output_dim = output_dim ##词袋大小
        self.num_layers = num_layers ##RNN层数
        self.dropout = dropout
 
        self.embedding = nn.Embedding(output_dim, embedding_dim)
        self.attention = Attention(hidden_dim)
        self.rnn = nn.GRU(embedding_dim + hidden_dim, hidden_dim,
                          num_layers=num_layers, dropout=dropout)
        self.out = nn.Linear(embedding_dim + hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(dropout)
 
    def forward(self, input, hidden, encoder_outputs):
        # input = [bsz]
        # hidden = [n layers * n directions, batch size, hid dim]
        # encoder_outputs = [sent len, batch size, hid dim * n directions]
        input = input.unsqueeze(0)
        # input = [1, bsz]
        embedded = self.dropout(self.embedding(input))
        # embedded = [1, bsz, emb dim]
        attn_weight = self.attention(hidden, encoder_outputs)
        # (batch_size, seq_len)
        context = attn_weight.unsqueeze(1).bmm(encoder_outputs.transpose(0, 1)).transpose(0, 1)
        # (batch_size, 1, hidden_dim * n_directions)
        # (1, batch_size, hidden_dim * n_directions)
        emb_con = torch.cat((embedded, context), dim=2)
        # emb_con = [1, bsz, emb dim + hid dim]
        _, hidden = self.rnn(emb_con, hidden)
        # outputs = [sent len, batch size, hid dim * n directions]
        # hidden = [n layers * n directions, batch size, hid dim]
        output = torch.cat((embedded.squeeze(0), hidden[-1], context.squeeze(0)), dim=1)
        output = F.log_softmax(self.out(output), 1)
        # outputs = [sent len, batch size, vocab_size]
        return output, hidden, attn_weight

In [ ]:
class Net(nn.Module):
    def __init__(self, encoder, decoder, device, teacher_forcing_ratio=0.5):
        super().__init__()
 
        self.encoder = encoder.to(device)
        self.decoder = decoder.to(device)
        self.device = device
        self.teacher_forcing_ratio = teacher_forcing_ratio
 
    def forward(self, src_seqs, src_lengths, trg_seqs):
        # src_seqs = [sent len, batch size]
        # trg_seqs = [sent len, batch size]
        batch_size = src_seqs.shape[1]
        max_len = trg_seqs.shape[0]
        trg_vocab_size = self.decoder.output_dim
        # tensor to store decoder outputs
        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)
        # hidden used as the initial hidden state of the decoder
        # encoder_outputs used to compute context
        encoder_outputs, hidden = self.encoder(src_seqs, src_lengths)
        # first input to the decoder is the <sos> tokens
        output = trg_seqs[0, :]
 
        for t in range(1, max_len): # skip sos
            output, hidden, _ = self.decoder(output, hidden, encoder_outputs)
            outputs[t] = output
            teacher_force = random.random() < self.teacher_forcing_ratio
            output = (trg_seqs[t] if teacher_force else output.max(1)[1])
        return outputs
 
    def predict(self, src_seqs, src_lengths, max_trg_len=30, start_ix=1):
        max_src_len = src_seqs.shape[0]
        batch_size = src_seqs.shape[1]
        trg_vocab_size = self.decoder.output_dim
        outputs = torch.zeros(max_trg_len, batch_size, trg_vocab_size).to(self.device)
        encoder_outputs, hidden = self.encoder(src_seqs, src_lengths)
        output = torch.LongTensor([start_ix] * batch_size).to(self.device)
        attn_weights = torch.zeros((max_trg_len, batch_size, max_src_len))
        for t in range(1, max_trg_len):
            output, hidden, attn_weight = self.decoder(output, hidden, encoder_outputs)
            outputs[t] = output
            output = output.max(1)[1]
            #attn_weights[t] = attn_weight
        return outputs, attn_weights

In [ ]:
en=Encoder(300,256) ##词向量维度300，rnn隐单元256
de=Decoder(len(word2id),300,256) ##词袋大小9132，词向量维度300，rnn隐单元256
network = Net(en,de,device) ##定义Seq2Seq实例
loss_fn = nn.CrossEntropyLoss() ##使用交叉熵损失函数
optimizer = optim.Adam(network.parameters(),lr=0.01) ##使用Adam优化器

In [ ]:
#Data Loader, (batch data)
#

In [ ]:
lowest_loss = 10
EPOCHS = 5
# 得到训练和测试的数据
for epoch in range(EPOCHS):
    network.train()
    
    # 得到训练和测试的数据
    x_train, y_train, x_test, y_test = data.next_batch(BATCH)  # 读取数据; shape:(sen_len,batch,embedding)
    #x_train shape: (batch,sen_len,embed_dim)
    #y_train shape: (batch,sen_len)
    batch_len = y_train.shape[0]
    #input_lengths = [30 for i in range(batch_len)] ## batch内每个句子的长度
    input_lengths = x_train[:,-1,0]
    input_lengths = input_lengths.tolist()
    #input_lengths = list(map(lambda x: int(x),input_lengths))
    input_lengths = [int(x) for x in input_lengths]
    y_lengths = y_train[:,-1]
    y_lengths = y_lengths.tolist()
    
    x_train = x_train[:,:-1,:] ## 除去长度信息
    x_train = torch.from_numpy(x_train) #shape:(batch,sen_len,embedding)
    x_train = x_train.float().to(device) 
    y_train = y_train[:,:-1] ## 除去长度信息
    y_train = torch.from_numpy(y_train) #shape:(batch,sen_len)
    y_train = torch.LongTensor(y_train)
    y_train = y_train.to(device) 
 
    seq_pairs = sorted(zip(x_train.contiguous(), y_train.contiguous(),input_lengths), key=lambda x: x[2], reverse=True)
    #input_lengths = sorted(input_lengths, key=lambda x: input_lengths, reverse=True)
    x_train, y_train,input_lengths = zip(*seq_pairs)
    x_train = torch.stack(x_train,dim=0).permute(1,0,2).contiguous()
    y_train = torch.stack(y_train,dim=0).permute(1,0).contiguous()
 
    outputs = network(x_train,input_lengths,y_train)
    
    #_, prediction = torch.max(outputs.data, 2)
        
    optimizer.zero_grad()
    outputs = outputs.float()
    # calculate the loss according to labels
    loss = loss_fn(outputs.view(-1, outputs.shape[2]), y_train.view(-1))
 
    # backward transmit loss
    loss.backward()
    # adjust parameters using Adam
    optimizer.step()
    print(loss)
 
    # 若测试准确率高于当前最高准确率，则保存模型
 
    if loss < lowest_loss:
        lowest_loss = loss
        model.save_model(network, MODEL_PATH, overwrite=True)
        print("step %d, best lowest_loss %g" % (epoch, lowest_loss))
    print(str(epoch) + "/" + str(args.EPOCHS))